In [1]:
import numpy as np 
import pandas as pd 
import math 
from sklearn import datasets, model_selection, naive_bayes
from sklearn.naive_bayes import MultinomialNB

Load datasets:

In [2]:
WildFiresData = pd.read_csv('Historical_Wildfires.csv')
HistoricalWeather = pd.read_csv('HistoricalWeather.csv')

Create train data and labels(it takes 7-10 minutes):

In [5]:
data_train = []
label_train = []
number_emptydata = 0
for i in range(len(WildFiresData)):
    dateoffire = WildFiresData['Date'].iloc[i]
    region = WildFiresData['Region'].iloc[i]
    spliteddate = dateoffire.split('/')
    if int(spliteddate[0]) < 10:
        spliteddate[0] = '0' + spliteddate[0]
    if int(spliteddate[1]) < 10:
        spliteddate[1] = '0' + spliteddate[1]
    finaldate = spliteddate[2] + '-' + spliteddate[1] + '-' + spliteddate[0]
    matchdate = HistoricalWeather[HistoricalWeather['Date'] == finaldate]
    final_matcheddate = matchdate[matchdate['Region'] == region]
    try:
        Precipitation = final_matcheddate[final_matcheddate['Parameter'] == 'Precipitation']['mean()'].iloc[0]
        RelativeHumidity = final_matcheddate[final_matcheddate['Parameter'] == 'RelativeHumidity']['mean()'].iloc[0]
        SoilWaterContent = final_matcheddate[final_matcheddate['Parameter'] == 'SoilWaterContent']['mean()'].iloc[0]
        SolarRadiation = final_matcheddate[final_matcheddate['Parameter'] == 'SolarRadiation']['mean()'].iloc[0]
        Temperature = final_matcheddate[final_matcheddate['Parameter'] == 'Temperature']['mean()'].iloc[0]
        WindSpeed = final_matcheddate[final_matcheddate['Parameter'] == 'WindSpeed']['mean()'].iloc[0]
        data_train.append([Precipitation, RelativeHumidity, SoilWaterContent, SolarRadiation, Temperature, WindSpeed])
        label_train.append(1)
    except:
        number_emptydata += 1 
    try:
        allregion = matchdate['Region'].drop_duplicates().values
        allregion = np.delete(allregion, np.where(allregion == region))
        for tempregion in allregion:
            final_matchedlabeldate = matchdate[matchdate['Region'] == tempregion]
            Precipitation = final_matchedlabeldate[final_matchedlabeldate['Parameter'] == 'Precipitation']['mean()'].iloc[0]
            RelativeHumidity = final_matchedlabeldate[final_matchedlabeldate['Parameter'] == 'RelativeHumidity']['mean()'].iloc[0]
            SoilWaterContent = final_matchedlabeldate[final_matchedlabeldate['Parameter'] == 'SoilWaterContent']['mean()'].iloc[0]
            SolarRadiation = final_matchedlabeldate[final_matchedlabeldate['Parameter'] == 'SolarRadiation']['mean()'].iloc[0]
            Temperature = final_matchedlabeldate[final_matchedlabeldate['Parameter'] == 'Temperature']['mean()'].iloc[0]
            WindSpeed = final_matchedlabeldate[final_matchedlabeldate['Parameter'] == 'WindSpeed']['mean()'].iloc[0]
            data_train.append([Precipitation, RelativeHumidity, SoilWaterContent, SolarRadiation, Temperature, WindSpeed])
            label_train.append(0)
    except:  
        number_emptydata += 1 

Use Machine Learning with trained data and labels to predict wildfire:

In [27]:
dangerlavel = str()

def FFWIformula(windavr, tempavr, havr):
    tempavr = (tempavr * (9/5)) + 32
    windavr = windavr/0.44704
    if havr < 10:
        m = 0.03229 + (0.281073 * havr)-(0.000578 * havr * tempavr)
    elif 10 <= havr <= 50:
        m = 2.22749 + (0.160107 * havr) - (0.01478 * tempavr)
    elif havr > 50:
        m = 21.0606 + (0.005565 * havr**2) - (0.00035 * havr * tempavr) - (0.483199 * havr)
    n = 1 - (2 * (m/30)) + (1.5 * (m/30)**2) - (0.5 * (m/30)**3)
    FFWI = n * (math.sqrt(1 + windavr**2)/0.3002)
    return round(FFWI, 2)

def machinelearing(Precipitation, RelativeHumidity, SoilWaterContent, SolarRadiation, Temperature, WindSpeed):
    model = MultinomialNB()
    X = data_train
    y = label_train
    model.fit(X, y)
    return round(model.predict_proba([[Precipitation, RelativeHumidity, SoilWaterContent, SolarRadiation, Temperature, WindSpeed]])[0][1]*100,2)

wind = float(input('mean wind (m/s):'))
temperature = float(input('mean temperature (C):'))
humidity = float(input('mean humidity (%):'))
precipitation = float(input('mean precipitation(in):'))
soilwatercontent = float(input('mean soil water content(m**3/m**-3):'))
solarradiation = float(input('mean solar radiation(MJ/day):'))

averageProb = (machinelearing(precipitation, humidity, soilwatercontent, solarradiation, temperature, wind) + FFWIformula(wind, temperature, humidity))/2
if averageProb < 50:
    dangerlavel = 'Low'
elif 50 <= averageProb < 70:
    dangerlavel = 'Medium'
elif 70 <= averageProb < 90:
    dangerlavel = 'High (dangerous!)'
elif 90 <= averageProb <= 100:
    dangerlavel = 'Extreme (Very dangerous!)'
elif 101 <= averageProb:
    dangerlavel = 'Very Extreme (Extremely dangerous!)'
    
print('Probability of occuring fire by Machine Learing(%): ', machinelearing(precipitation, humidity, soilwatercontent, solarradiation, temperature, wind))
print('Fire index by FFWI formula: ', FFWIformula(wind, temperature, humidity))
print('Danger level by average probability of FFWI formula and ML: ', dangerlavel)

mean wind (m/s):11
mean temperature (C):40
mean humidity (%):5
mean precipitation(in):0.04
mean soil water content(m**3/m**-3):0.1
mean solar radiation(MJ/day):30
Probability of occuring fire by Machine Learing(%):  99.08
Fire index by FFWI formula:  75.99
Danger level by average probability of FFWI formula and ML:  High (dangerous!)
